In [25]:
!pip install tensorflow

     |████████████████████████████████| 207.1 MB 41 kB/s  eta 0:00:01     |███████████████                 | 97.7 MB 5.7 MB/s eta 0:00:20     |███████████████▏                | 98.0 MB 5.7 MB/s eta 0:00:20     |███████████████████             | 122.6 MB 7.5 MB/s eta 0:00:12     |█████████████████████████████▉  | 193.4 MB 4.7 MB/s eta 0:00:03
  Using cached protobuf-3.19.1-cp38-cp38-macosx_10_9_x86_64.whl (1.0 MB)
     |████████████████████████████████| 126 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 974 kB/s eta 0:00:01
     |████████████████████████████████| 57 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 8.4 MB/s eta 0:00:01     |█████████████████████████████▉  | 11.4 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 7.9 MB/s eta 0:00:01
     |████████████████████████████████|

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import random
import math
print('finished importing')

finished importing


In [3]:
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf

In [4]:
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

In [5]:
# from google.colab import drive
# drive.mount('/content/drive/')
# DIR = '/content/drive/MyDrive/Team_34_Algo_Project'
DIR = '.'

In [16]:
ratings_df = pd.read_csv(f'{DIR}/data/rating.csv')
ratings_df.columns =['user_id', 'movie_id', 'rating', 'unix_timestamp']
# movie_df = pd.read_csv(f'{DIR}/data/movie.csv')

# df = movie_df.merge(ratings_df, on="movieId")
# df.head()
ratings_df

,user_id,movie_id,rating,unix_timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [9]:
print(ratings_df.shape)
print(movie_df.shape)

(20000263, 4)
(27278, 3)


In [10]:
movie_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [11]:
ratings_df

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [12]:
#20 million ratings and 465,000 tag applications applied to 27,000 movies by 138,000 users.
print(f"{pd.unique(ratings_df['movieId']).shape} vs {movie_df.shape}")

(26744,) vs (27278, 3)


In [22]:
ratings_df['movie_id'].max()

131262

In [13]:
len(ratings_df['movieId'].unique())

26744

In [14]:
movies = pd.DataFrame(ratings_df['movieId'], columns=['movieId'])
movies

,movieId
0,2
1,29
2,32
3,47
4,50
...,...
20000258,68954
20000259,69526
20000260,69644
20000261,70286


In [15]:
#20 million ratings and 465,000 tag applications applied to 27,000 movies by 138,000 users.
pd.unique(ratings_df['userId']).shape

(138493,)

In [16]:
users = pd.DataFrame(ratings_df['userId'], columns=['userId'])
users

,userId
0,1
1,1
2,1
3,1
4,1
...,...
20000258,138493
20000259,138493
20000260,138493
20000261,138493


In [17]:
users.shape

(20000263, 1)

In [18]:
users_ratings = (
    ratings_df
    .groupby('userId', as_index=False)
    .agg({'rating': ['count', 'mean']})
    .flatten_cols()
    .merge(users, on='userId')
)
users_ratings

,userId,rating count,rating mean
0,1,175,3.742857
1,1,175,3.742857
2,1,175,3.742857
3,1,175,3.742857
4,1,175,3.742857
...,...,...,...
20000258,138493,373,4.172922
20000259,138493,373,4.172922
20000260,138493,373,4.172922
20000261,138493,373,4.172922


In [19]:
def build_rating_sparse_tensor(ratings_df):
  """
  Args:
    ratings_df: a pd.DataFrame with `user_id`, `movie_id` and `rating` columns.
  Returns:
    a tf.SparseTensor representing the ratings matrix.
  """
  indices = ratings_df[['userId', 'movieId']].values
  vals = ratings_df['rating'].values
  return tf.SparseTensor(
      indices=indices,
      values=vals,
      dense_shape=[users.shape[0], movies.shape[0]]) 

In [20]:
#@title Alternate Solution
def sparse_mean_square_error(sparse_ratings, user_embeddings, movie_embeddings):
  """
  Args:
    sparse_ratings: A SparseTensor rating matrix, of dense_shape [N, M]
    user_embeddings: A dense Tensor U of shape [N, k] where k is the embedding
      dimension, such that U_i is the embedding of user i.
    movie_embeddings: A dense Tensor V of shape [M, k] where k is the embedding
      dimension, such that V_j is the embedding of movie j.
  Returns:
    A scalar Tensor representing the MSE between the true ratings and the
      model's predictions.
  """
  predictions = tf.reduce_sum(
      tf.gather(user_embeddings, sparse_ratings.indices[:, 0]) *
      tf.gather(movie_embeddings, sparse_ratings.indices[:, 1]),
      axis=1)
  loss = tf.losses.mean_squared_error(sparse_ratings.values, predictions)
  return loss

In [21]:
# @title CFModel helper class (run this cell)
class CFModel(object):
  """Simple class that represents a collaborative filtering model"""
  def __init__(self, embedding_vars, loss, metrics=None):
    """Initializes a CFModel.
    Args:
      embedding_vars: A dictionary of tf.Variables.
      loss: A float Tensor. The loss to optimize.
      metrics: optional list of dictionaries of Tensors. The metrics in each
        dictionary will be plotted in a separate figure during training.
    """
    self._embedding_vars = embedding_vars
    self._loss = loss
    self._metrics = metrics
    self._embeddings = {k: None for k in embedding_vars}
    self._session = None

  @property
  def embeddings(self):
    """The embeddings dictionary."""
    return self._embeddings

  def train(self, num_iterations=100, learning_rate=1.0, plot_results=True,
            optimizer=tf.train.GradientDescentOptimizer):
    """Trains the model.
    Args:
      iterations: number of iterations to run.
      learning_rate: optimizer learning rate.
      plot_results: whether to plot the results at the end of training.
      optimizer: the optimizer to use. Default to GradientDescentOptimizer.
    Returns:
      The metrics dictionary evaluated at the last iteration.
    """
    with self._loss.graph.as_default():
      opt = optimizer(learning_rate)
      train_op = opt.minimize(self._loss)
      local_init_op = tf.group(
          tf.variables_initializer(opt.variables()),
          tf.local_variables_initializer())
      if self._session is None:
        self._session = tf.Session()
        with self._session.as_default():
          self._session.run(tf.global_variables_initializer())
          self._session.run(tf.tables_initializer())
          tf.train.start_queue_runners()

    with self._session.as_default():
      local_init_op.run()
      iterations = []
      metrics = self._metrics or ({},)
      metrics_vals = [collections.defaultdict(list) for _ in self._metrics]

      # Train and append results.
      for i in range(num_iterations + 1):
        _, results = self._session.run((train_op, metrics))
        if (i % 10 == 0) or i == num_iterations:
          print("\r iteration %d: " % i + ", ".join(
                ["%s=%f" % (k, v) for r in results for k, v in r.items()]),
                end='')
          iterations.append(i)
          for metric_val, result in zip(metrics_vals, results):
            for k, v in result.items():
              metric_val[k].append(v)

      for k, v in self._embedding_vars.items():
        self._embeddings[k] = v.eval()

      if plot_results:
        # Plot the metrics.
        num_subplots = len(metrics)+1
        fig = plt.figure()
        fig.set_size_inches(num_subplots*10, 8)
        for i, metric_vals in enumerate(metrics_vals):
          ax = fig.add_subplot(1, num_subplots, i+1)
          for k, v in metric_vals.items():
            ax.plot(iterations, v, label=k)
          ax.set_xlim([1, num_iterations])
          ax.legend()
      return results

In [22]:
#@title Solution
def build_model(ratings, embedding_dim=3, init_stddev=1.):
  """
  Args:
    ratings: a DataFrame of the ratings
    embedding_dim: the dimension of the embedding vectors.
    init_stddev: float, the standard deviation of the random initial embeddings.
  Returns:
    model: a CFModel.
  """
  # Split the ratings DataFrame into train and test.
  train_ratings, test_ratings = split_dataframe(ratings)
  # SparseTensor representation of the train and test datasets.
  A_train = build_rating_sparse_tensor(train_ratings)
  A_test = build_rating_sparse_tensor(test_ratings)
    
  print('A_train shape', A_train.shape)
  print('A_test shape', A_test.shape)

  # Initialize the embeddings using a normal distribution.
  U = tf.Variable(tf.random_normal(
      [A_train.dense_shape[0], embedding_dim], stddev=init_stddev))
  V = tf.Variable(tf.random_normal(
      [A_train.dense_shape[1], embedding_dim], stddev=init_stddev))
  
  train_loss = sparse_mean_square_error(A_train, U, V)
  test_loss = sparse_mean_square_error(A_test, U, V)
  metrics = {
      'train_error': train_loss,
      'test_error': test_loss
  }
  embeddings = {
      "userId": U,
      "movieId": V
  }
  return CFModel(embeddings, train_loss, [metrics])

In [23]:
# Utility to split the data into training and test sets.
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

In [ ]:
# Build the CF model and train it.
model = build_model(ratings_df, embedding_dim=30, init_stddev=0.5)
model.train(num_iterations=1000, learning_rate=10.)

A_train shape (20000263, 20000263)
A_test shape (20000263, 20000263)


In [12]:
rating_df.describe()

,userId,movieId,rating
count,2.000026e+07,2.000026e+07,2.000026e+07
mean,6.904587e+04,9.041567e+03,3.525529e+00
std,4.003863e+04,1.978948e+04,1.051989e+00
min,1.000000e+00,1.000000e+00,5.000000e-01
25%,3.439500e+04,9.020000e+02,3.000000e+00
50%,6.914100e+04,2.167000e+03,3.500000e+00
75%,1.036370e+05,4.770000e+03,4.000000e+00
max,1.384930e+05,1.312620e+05,5.000000e+00


In [9]:
movie_df.describe()

,movieId
count,27278.000000
mean,59855.480570
std,44429.314697
min,1.000000
25%,6931.250000
50%,68068.000000
75%,100293.250000
max,131262.000000


In [21]:
len(pd.unique(df['userId']))

138493

In [5]:
title_counts = pd.DataFrame(df["title"].value_counts())

In [6]:
# # remove movies that appear less 3000 times
# popular_threashold = 2000
# popular_movies = df[~df["title"].isin(title_counts[title_counts["title"] <= popular_threashold].index)]

In [8]:
user_movie_table = movies_df.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_table.head()

NameError: name 'movies_df' is not defined

In [23]:
user_movie_table.shape

(138493, 2034)

In [7]:
random_user = int(pd.Series(user_movie_table.index).sample(1, random_state = 200).values)
print("The random user is " + str(random_user))
random_user_df = user_movie_table[user_movie_table.index == random_user]
random_user_movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

NameError: name 'user_movie_table' is not defined

In [ ]:
movies_watched_df = user_movie_table[random_user_movies_watched]
movies_watched_df.head()

In [ ]:
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count.head(25)
user_movie_count = user_movie_count.reset_index() 
user_movie_count.columns = ["userId", "movie_count"]
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 20]["userId"]
users_same_movies.index


final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[random_user_movies_watched]])
final_df.head()

In [ ]:
corr_table = final_df.T.corr(method ='kendall').unstack()
corr_table = corr_table.sort_values()
corr_table

In [ ]:
#calculate the similarities scores using .corr()

corr_df = pd.DataFrame(corr_table, columns=["corr"])
corr_df.head()
corr_df.index.names = ['target_user', 'predict_user']
corr_df = corr_df.reset_index()
corr_df.sort_values(by='corr',ascending=False).head(100)

In [ ]:
corr_threshold = 0.5
top_users = corr_df[(corr_df["target_user"] == random_user) & (corr_df["corr"] > corr_threshold)][
    ["predict_user", "corr"]].reset_index(drop=True)
top_users.head()
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"predict_user": "userId"}, inplace=True)
top_users.head()

In [ ]:
# calcuating weighted_rating use the similarity score
top_users_ratings = top_users.merge(rating_df[["userId", "movieId", "rating"]], how='inner')
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head(100)

In [ ]:
random_movieIds = top_users_ratings['movieId'].sample(10).values

In [ ]:
random_movieIds

In [ ]:
target_ratings, predicted_movie_ratings = [], []

for i in random_movieIds:
  # append results
  current = top_users_ratings.loc[top_users_ratings["movieId"].isin(random_movieIds)]
  target_ratings.append(current['rating'])
  predicted_movie_ratings.append(current['weighted_rating'])

In [ ]:
# compute MSE
from sklearn.metrics import mean_squared_error
mse_batch = mean_squared_error(np.array(target_ratings), np.array(predicted_movie_ratings))

print(mse_batch)

In [ ]:
# compute RMSE
rmse_batch = mean_squared_error(np.array(target_ratings), np.array(predicted_movie_ratings), squared=False)

print(rmse_batch)

# Overall MSE

In [ ]:
# Calculate MSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(top_users_ratings['rating'], top_users_ratings['weighted_rating'])

In [ ]:
print(mse)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(top_users_ratings['rating'], top_users_ratings['weighted_rating'], squared=False)

In [ ]:
print(rmse)